# 01_walkflow
* [1. Choose a model](#1.-Choose-a-model)
* [2. Optimize your model](#2.-Optimize-your-model)
* [3. Convert the model](#3.-Convert-the-model)
* [4. Inference the model](#4.-Inference-the-model)

In [1]:
import numpy as np
import tensorflow as tf

## 1. Choose a model

 * h5 file
 * SavedModel

In [2]:
# Build a pre-trained model: h5 file, SavedModel
from tensorflow.keras.applications.mobilenet import MobileNet
model = MobileNet(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

# Save to h5 file
model.save('model/mobilenet_keras_model.h5') 

# Save to SavedModel directory
tf.saved_model.save(model, 'model/mobilenet_saved_model') 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model/mobilenet_saved_model/assets


## 2. Optimize your model
 * Post-training (請參考 02_post_training.ipynb)
     - Quantization
 * Aware-training (請參考 03_aware_training.ipynb)
     - Quantization
     - Pruning
     - Clustering 

## 3. Convert the model

### Command Line Tool
 --output_file OUTPUT_FILE <br>
<font color="red">&emsp;Full filepath of the output file.</font><br>
 --saved_model_dir SAVED_MODEL_DIR <br>
<font color="red">&emsp;Full path of the directory containing the SavedModel.</font><br>
 --keras_model_file KERAS_MODEL_FILE <br>
<font color="red">&emsp;Full filepath of HDF5 file containing tf.Keras model.</font><br>

In [3]:
!tflite_convert --help

usage: tflite_convert [-h] --output_file OUTPUT_FILE
                      (--saved_model_dir SAVED_MODEL_DIR | --keras_model_file KERAS_MODEL_FILE)

Command line tool to run TensorFlow Lite Converter.

optional arguments:
  -h, --help            show this help message and exit
  --output_file OUTPUT_FILE
                        Full filepath of the output file.
  --saved_model_dir SAVED_MODEL_DIR
                        Full path of the directory containing the SavedModel.
  --keras_model_file KERAS_MODEL_FILE
                        Full filepath of HDF5 file containing tf.Keras model.


### Converts a Keras model with Command Line Tool

In [4]:
!tflite_convert \
  --keras_model_file=model/mobilenet_keras_model.h5 \
  --output_file=tflite_model/mobilenet.tflite

2020-10-28 11:33:37.625403: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-28 11:33:37.674910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
pciBusID: 0000:04:00.0
2020-10-28 11:33:37.690820: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
pciBusID: 0000:0c:00.0
2020-10-28 11:33:37.693423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
pciBusID: 0000:0e:00.0
2020-10-28 11:33:37.696047: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 3 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
pciBusID: 0000:0f:00.0
2020-10

### Converts a SavedModel with Command Line Tool

In [5]:
!tflite_convert \
  --saved_model_dir=model/mobilenet_saved_model  \
  --output_file=tflite_model/mobilenet.tflite

2020-10-28 11:33:47.580393: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-28 11:33:47.613521: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
pciBusID: 0000:04:00.0
2020-10-28 11:33:47.615685: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
pciBusID: 0000:0c:00.0
2020-10-28 11:33:47.617832: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 2 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
pciBusID: 0000:0e:00.0
2020-10-28 11:33:47.619970: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 3 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.582
pciBusID: 0000:0f:00.0
2020-10

### Python API
Convert a TensorFlow 2.x model using TensorFlow Lite converter: tf.lite.TFLiteConverter.
 * Converts a Keras model
     - tf.lite.TFLiteConverter.from_keras_model()
 * Converts a SavedModel
     - tf.lite.TFLiteConverter.from_saved_model() (recommended)
 * Converts concrete functions
     - tf.lite.TFLiteConverter.from_concrete_functions()

### Converts a Keras model with Python API

In [6]:
# Load the model.
model = tf.keras.models.load_model('model/mobilenet_keras_model.h5')

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('tflite_model/mobilenet.tflite', 'wb') as f:
    f.write(tflite_model)

### Converts a SavedModel with Python API

In [7]:
# path to the SavedModel directory
saved_model_dir = 'model/mobilenet_saved_model'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the model.
with open('tflite_model/mobilenet.tflite', 'wb') as f:
    f.write(tflite_model)

### Converts concrete functions with Python API

In [8]:
# Create a model using low-level tf.* APIs
class Squared(tf.Module):
    @tf.function
    def __call__(self, x):
        return tf.square(x)
model = Squared()
concrete_func = model.__call__.get_concrete_function(
    tf.TensorSpec(shape=[None, 1], dtype=tf.float32)
)

In [9]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()
# Save the model.
with open('tflite_model/concrete_functions.tflite', 'wb') as f:
    f.write(tflite_model)

## 4. Inference the model

In [10]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="tflite_model/mobilenet.tflite")
interpreter.allocate_tensors()

In [11]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [12]:
input_details

[{'name': 'input_1',
  'index': 101,
  'shape': array([  1, 224, 224,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

In [13]:
output_details

[{'name': 'Identity',
  'index': 0,
  'shape': array([   1, 1000], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

In [14]:
# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

In [15]:
# The function 'tensor()' returns a pointer to the tensor.
output_data = interpreter.tensor(output_details[0]['index'])
print(output_data)

<function Interpreter.tensor.<locals>.<lambda> at 0x7f6c30ac3d08>


In [16]:
# The function 'get_tensor()' returns a copy of the tensor data.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[2.17110355e-05 8.06244061e-05 6.94543472e-04 2.59275275e-05
  1.04193052e-03 4.06472333e-04 1.92650768e-03 1.44019014e-05
  4.60201627e-05 1.37082927e-04 1.58209703e-04 4.67627069e-05
  3.87580396e-04 5.21293026e-04 6.67030516e-04 2.20626662e-03
  5.20122063e-04 1.64278230e-04 4.05233307e-03 1.40845863e-04
  7.00320015e-05 2.32152129e-03 1.08423072e-03 2.51309248e-03
  1.23647624e-03 2.81769080e-05 4.13653870e-05 3.17835889e-04
  1.90152969e-05 8.90031515e-05 1.68984152e-05 1.33654787e-04
  1.75177975e-05 2.82391877e-04 2.06027972e-03 1.19738843e-05
  2.29089546e-05 1.66057522e-04 4.42326418e-05 1.21327656e-04
  1.10612775e-04 2.06580051e-04 3.65981432e-05 2.50309895e-05
  1.57548720e-03 2.30968672e-05 6.74407114e-04 1.03858720e-05
  3.90985442e-05 3.44726359e-05 4.57640650e-04 1.74257286e-06
  2.65230890e-04 3.77055170e-04 6.00597086e-05 1.54974638e-04
  4.14949027e-05 5.20142639e-05 4.63964330e-04 8.94496698e-05
  6.70885638e-05 4.49067784e-06 7.25236077e-06 7.48027060e-06
  8.2629